In [3]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

True

In [8]:
spotify_id = os.getenv('SPOTIFY_ID')
spotify_key = os.getenv('SPOTIFY_TOKEN')

In [42]:
import requests
from base64 import b64encode

def get_spotify_token(client_id, client_secret):
    # Encode as Base64
    credentials = b64encode(f"{client_id}:{client_secret}".encode()).decode('utf-8')

    headers = {
        "Authorization": f"Basic {credentials}"
    }

    body = {
        "grant_type": "client_credentials"
    }

    token_url = "https://accounts.spotify.com/api/token"
    response = requests.post(token_url, headers=headers, data=body)
    response_data = response.json()
    return response_data['access_token']

access_token = get_spotify_token(spotify_id, spotify_key)

In [26]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
id="11dFghVXANMlKmJXsNCbNl"
endpoint = f"https://api.spotify.com/v1/audio-features/{id}"

In [16]:
response = requests.get(endpoint, headers=headers)


In [17]:
response.json()

{'danceability': 0.696,
 'energy': 0.905,
 'key': 2,
 'loudness': -2.743,
 'mode': 1,
 'speechiness': 0.103,
 'acousticness': 0.011,
 'instrumentalness': 0.000905,
 'liveness': 0.302,
 'valence': 0.625,
 'tempo': 114.944,
 'type': 'audio_features',
 'id': '11dFghVXANMlKmJXsNCbNl',
 'uri': 'spotify:track:11dFghVXANMlKmJXsNCbNl',
 'track_href': 'https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/11dFghVXANMlKmJXsNCbNl',
 'duration_ms': 207960,
 'time_signature': 4}

In [71]:
import time
import requests

def safe_request(url:str, headers:dict, params:dict=None, max_retries:int=20):
    retry_wait = 1
    for i in range(max_retries):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', retry_wait))
            print(retry_after)
            print(f"Rate limit exceeded, retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            retry_wait *= 2 
        else:
            response.raise_for_status()
    raise Exception("Max retries exceeded")

def search_tracks_by_genre(access_token:str, genre:str, limit:int=50, records:int=500):
    url = "https://api.spotify.com/v1/search"
    track_ids = []
    offset = 0

    while len(track_ids) < records:
        query_params = {
            "q": f"genre:{genre}",
            "type": "track",
            "limit": limit,
            "offset": offset
        }
        headers = {"Authorization": f"Bearer {access_token}"}
        
        response = safe_request(url, headers=headers, params=query_params)
        data = response.json()
        tracks = data.get('tracks', {}).get('items', [])
        
        if not tracks:
            break

        for track in tracks:
            track_ids.append(track['id'])
        
        offset += limit
        if len(tracks) < limit:
            break

    return track_ids

def get_audio_features_by_id(access_token, track_ids):
    audio_features = []
    batch_size = 100

    for start in range(0, len(track_ids), batch_size):
        end = start + batch_size
        batch_ids = track_ids[start:end]
        url = f"https://api.spotify.com/v1/audio-features?ids={','.join(batch_ids)}"
        headers = {"Authorization": f"Bearer {access_token}"}
        
        print(f"{url=}")
        response = safe_request(url, headers=headers)
        data = response.json()

        if 'error' in data:
            print("Error:", data['error']['message'])
            return None
        
        batch_features = data.get('audio_features', [])
        if batch_features:
            audio_features.extend(batch_features)
        else:
            print("No data received for batch:", batch_ids)

    return audio_features


In [65]:
import json

with open("music_genres.json", "r") as af:
    genres = json.load(af)

In [66]:
complete_track_list = []

for genre in genres["music_genres"][0:4]: # add limit to make this a simple initial task
    complete_track_list.extend(search_tracks_by_genre(access_token, genre))

In [67]:
with open("track_ids.json", "w") as af:
    af.write(json.dumps({"track_ids": complete_track_list}))

In [68]:
with open("track_ids.json", "r") as af:
    track_list = json.load(af)

In [69]:
complete_track_list = track_list["track_ids"]

In [72]:
# Rate limit on this endpoint is crazy
audio_feature_list = get_audio_features_by_id(access_token, complete_track_list)

url='https://api.spotify.com/v1/audio-features?ids=2bQarhW0eXpUO2g5a4ncZp,6Y1sT1W7ATUhgIyuB6fd2P,7xKPrsvnyVn0fEDmHHV4SF,47NXyPVI1qgz5ThcyiBM8I,5YWQlCqgPpyeZ17a9eqgZa,7M62Hqghz8osQILzAIJWaP,69S4fitMHVrVgbk8WZcPsZ,0y3ZRDYRDDMrbatpvhqZ1r,65P3IBgRj9aO0wQfmXzxW1,1pjwVLK6IE5oAZ9WSmlmm9,4AFb7GPWT2osksulSs7qjf,6rwsduqKvGn30R3J1FL6hK,4tFiDHBcVUkFWW5wc7Ry9M,7sOuqN865ZVObRS3LfwHzs,0UFeSZvYbUPZVeHCo3SNF3,0OJ6vKdJ71hAeqfzZ701pQ,7MsbXbQN2xc4NSuodDqIc4,7sblnXksCahIgUAdX1aTPy,7KO2Lk9X5Eb4RIaUutW5DL,1otUiIpY2QekoZwcPJtVbL,5qFl7PkUlDijgWksSyFIFg,4NSYEUdBPPBjIaPHIMPqdN,2Fr0zC0wKX01X7LSOMeVLr,6mtLysLyNyfWbr58nfqMV1,4ZiZDc0NIrrGTv3SNOMqZ3,3iulr03RqyQBDJItsaR46e,5f09cMt7yfm8DPKAHRFOXS,05XhneYOozrRUuw3hdaYFu,6VFcqDx6hFv6lSylQaNr42,5cgbLlOQNOQZkU5af0Aict,1hecBi6qXM8bjTPnJjr205,3o0pDrsaLubIoRSSeS3fTO,1RaUezq8CWuRubSda1HaLP,5qVPF3mjUkGSLda9ISeWjF,7zxym1sT9lcnmQ8ojqSqc4,33xo5xX9T7EQSAGsFfhBlH,4SepgeUrKPOwvpURr0aHMP,5362HePURJTQuNtFjyDmuM,0rGRq785D2i84ToyKgQMBR,7vLHGc0XlJoSmaOV4Grwql,6eWIJTgrEBJPs5quLHwde0,0Z5mEG

In [56]:
with open("audio_feature.json", "w") as af:
    af.write(json.dumps({"audio_features": audio_feature_list}))

In [57]:
audio_feature_list

[]